In [1]:
import numpy as np
import h5py
import math
from sklearn.decomposition import PCA
from pca import pca
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm
import peakutils
import os, sys
from BaselineRemoval import BaselineRemoval
from sklearn.cluster import DBSCAN
import scipy
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [2]:
class HiddenPrints:
	def __enter__(self):
		self._original_stdout = sys.stdout
		sys.stdout = open(os.devnull, 'w')

In [3]:
def __exit__(self, exc_type, exc_val, exc_tb):
	sys.stdout.close()
	sys.stdout = self._original_stdout

In [4]:
def remove_outliers(xset, yset, zset, eset, threshold):
	"""
	Uses DBSCAN to find and remove outliers in 3D data
	"""
	data = np.array([xset.T, yset.T, zset.T]).T
	DBSCAN_cluster = DBSCAN(eps=7, min_samples=10).fit(data)
	out_of_cluster_index = np.where(DBSCAN_cluster.labels_==-1)
	del data
	rev = out_of_cluster_index[0][::-1]
	#if len(out_of_cluster_index[0]) > 0:
	for i in rev:
		xset = np.delete(xset, i)
		yset = np.delete(yset, i)
		zset = np.delete(zset, i)
		eset = np.delete(eset, i)
	#if len(xset) <= threshold:
	#	veto = True
	#else:
    #veto = False
	return xset, yset, zset, eset, False

In [5]:
print('Veto on Length @ 70 mm')
print('Veto on Integrated Charge @ 800k')
def track_len(xset, yset, zset):
    """
    Uses PCA to find the length of a track
    """
    veto_on_length = False
 
    # Form data matrix
    data = np.concatenate((xset[:, np.newaxis], 
                           yset[:, np.newaxis], 
                           zset[:, np.newaxis]), 
                           axis=1)

    # Use PCA to find track length
    pca = PCA(n_components=3)
    principalComponents = pca.fit(data)
    principalComponents = pca.transform(data)
    principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
    
    track_len = 2.35*principalDf.std()[0]
    track_width = 2.35*principalDf.std()[1]
    track_depth = 2.35*principalDf.std()[2]
    #if track_len > 70:
    #    veto_on_length = True
    
    return track_len, veto_on_length, track_width, track_depth

Veto on Length @ 70 mm
Veto on Integrated Charge @ 800k


In [6]:
def main(h5file, threshold):
    """
    This functions does the following: 
    - Converts h5 files into ndarrays. 
    - Removes outliers.
    - Calls PCA to return track length.
    - Sums mesh signal to return energy.
    """
    # Converts h5 files into ndarrays, and output each event dataset as a separte list
    num_events = int(len(list(h5file.keys()))/2) 
    
    len_list = []
    width_list = []
    depth_list = []
    good_events = []
    tot_energy = []
    tracemax_list = []
    tracedev_list = []
    pads_list = []
    
    skipped_events = 0
    veto_events = 0
    
    pbar = tqdm(total=num_events+1)
    for i in range(0, num_events):
        str_event = f"Event_[{i}]"
        
        # Apply pad threshold
        event = np.array(h5file[str_event][:])
        if len(event) <= threshold:
            skipped_events += 1
            pbar.update(n=1)
            continue
            
        # Make copy of datasets
        dset_0_copyx = event['x']
        dset_0_copyy = event['y'] 
        dset_0_copyz = event['z'] - min(event['z'])
        dset_0_copye = event['A']

        
        # Apply veto condition
        R = 36                           # Radius of the pad plane
        r = np.sqrt(dset_0_copyx**2 + dset_0_copyy**2)
        statements = np.greater(r, R)    # Check if any point lies outside of R
      
        if np.any(statements) == True:
            veto_events += 1
            pbar.update(n=1)
            continue
        
        
        # Call remove_outliers to get dataset w/ outliers removed
        dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, veto = remove_outliers(dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, threshold)
        if veto == True:
            skipped_events += 1
            pbar.update(n=1)
            continue

        
        # Call track_len() to create lists of all track lengths
        length, veto_on_length, width, depth = track_len(dset_0_copyx, dset_0_copyy, dset_0_copyz)
        if veto_on_length == True:
            veto_events += 1
            pbar.update(n=1)
            continue

        
       	str_trace = f"Trace_[{i}]"
        trace = np.array(h5file[str_trace][:])
        max_val = np.argmax(trace)
        low_bound = max_val - 75
        if low_bound < 0:
            low_bound = 5
        upper_bound = max_val + 75
        if upper_bound > 511:
            upper_bound = 506
        trace = trace[low_bound:upper_bound]

        polynomial_degree=2 
        baseObj=BaselineRemoval(trace)
        trace=baseObj.IModPoly(polynomial_degree)

        #if np.sum(trace) > 800000:
        #    veto_events += 1
        #    pbar.update(n=1)
        #    continue

        len_list.append(length)
        width_list.append(width)
        depth_list.append(depth)
        tot_energy.append(np.sum(trace))
        tracemax_list.append(np.max(trace))
        tracedev_list.append(np.std(trace))
        pads_list.append(len(trace[trace > 0]))

        # Track event number of good events
        good_events.append(i)  
        pbar.update(n=1)

    return (tot_energy, skipped_events, veto_events, good_events, len_list, width_list, depth_list, tracemax_list, tracedev_list, pads_list)

In [7]:
# assign directory
directory = "C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/"

output_df = pd.DataFrame(columns=['file','event','length', 'width', 'depth', 'tracesum', 'tracemax', 'tracedev', 'padnum'])

file_names = []
# iterate over files in directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        if f[-3:] == '.h5':
            print(f)
            file_names.append(filename)
            h5f = h5py.File(directory+filename, 'r')
            (tot_energy, skipped_events, veto_events, good_events, len_list, width_list, depth_list, tracemax_list, tracedev_list, pads_list) = main(h5file=h5f, threshold=15)
            for event in range(len(tot_energy)):
                output_df = output_df.append({'file' : filename, 
                                              'event' : event, 
                                              'length' : len_list[event], 
                                              'width' : width_list[event], 
                                              'depth' : depth_list[event], 
                                              'tracesum' : tot_energy[event], 
                                              'tracemax' : tracemax_list[event], 
                                              'tracedev' : tracedev_list[event], 
                                              'padnum' : pads_list[event]}
                                             ,ignore_index = True)


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/273_800p.h5


100%|█████████▉| 676/677 [00:13<00:00, 50.73it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning2protons0-800p.h5


 98%|█████████▊| 50/51 [00:01<00:00, 42.86it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning2protons1-800p.h5


...

 98%|█████████▊| 50/51 [00:01<00:00, 41.71it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning2protons98-800p.h5


 98%|█████████▊| 50/51 [00:01<00:00, 39.06it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning2protons99-800p.h5


 98%|█████████▊| 50/51 [00:01<00:00, 44.52it/s]


In [8]:
for file in file_names:
    print(file)
    print('length:\t', output_df[output_df['file'] == file]['length'].mean(),'\t', output_df[output_df['file'] == file]['length'].std())
    print('width:\t', output_df[output_df['file'] == file]['width'].mean(),'\t', output_df[output_df['file'] == file]['width'].std())
    print('depth:\t', output_df[output_df['file'] == file]['depth'].mean(),'\t', output_df[output_df['file'] == file]['depth'].std())
    print('tracesum:\t', output_df[output_df['file'] == file]['tracesum'].mean(),'\t', output_df[output_df['file'] == file]['tracesum'].std())
    print('tracemax:\t', output_df[output_df['file'] == file]['tracemax'].mean(),'\t', output_df[output_df['file'] == file]['tracemax'].std())
    print('tracedev:\t', output_df[output_df['file'] == file]['tracedev'].mean(),'\t', output_df[output_df['file'] == file]['tracedev'].std())
    print('padnum:\t', output_df[output_df['file'] == file]['padnum'].mean(),'\t', output_df[output_df['file'] == file]['padnum'].std())
    print()

273_800p.h5
length:	 13.598839169264545 	 0.36378981766204926
width:	 5.490411630687277 	 1.079046393352792
depth:	 1.1956686193850883 	 0.4627512586642447
tracesum:	 155282.26997909456 	 1353.2815542925161
tracemax:	 17261.59640386621 	 3820.7164201418886
tracedev:	 3396.80817140803 	 395.6029936835511
padnum:	 89.44822485207101 	 4.124217163844774

tuning2protons0-800p.h5
length:	 15.249090908108828 	 1.1433127793866278
width:	 9.331519885486022 	 1.0410393374235865
depth:	 3.7517142148937 	 0.9337680758007404
tracesum:	 301306.6144510945 	 1972.7452318870767
tracemax:	 29260.05320781326 	 3517.8997633018234
tracedev:	 6142.8029665961485 	 414.6603133665401
padnum:	 109.52 	 7.918565114298661

tuning2protons1-800p.h5
length:	 15.173485044746657 	 1.2212138085565611
width:	 9.103423780345384 	 1.009437556731119
depth:	 3.8070539756262587 	 0.8921949164556146
tracesum:	 329464.87619933515 	 1865.3789467756646
tracemax:	 31677.92808539405 	 3675.2804721654447
tracedev:	 6683.26971266091

In [9]:
output_df.to_csv('event summary.csv', index=False)

In [16]:
output_df = pd.read_csv('event summary.csv')
file_names = output_df['file'].unique() 

run_df = pd.DataFrame(columns=['file','length', 'width', 'depth', 'tracesum', 'tracemax', 'tracedev', 'padnum'])

reference_run = '273_800p.h5'

for file in file_names:
    run_df = run_df.append({'file' : file, 
                            'length' : output_df[output_df['file'] == file]['length'].mean(), 
                            'width' : output_df[output_df['file'] == file]['width'].mean(), 
                            'depth' : output_df[output_df['file'] == file]['depth'].mean(), 
                            'tracesum' : output_df[output_df['file'] == file]['tracesum'].mean(), 
                            'tracemax' : output_df[output_df['file'] == file]['tracemax'].mean(), 
                            'tracedev' : output_df[output_df['file'] == file]['tracedev'].mean(), 
                            'padnum' : output_df[output_df['file'] == file]['padnum'].mean()}
                           ,ignore_index = True)

run_df['length'] = (run_df['length'] - output_df[output_df['file'] == reference_run]['length'].mean()) / output_df[output_df['file'] == reference_run]['length'].std()
run_df['width'] = (run_df['width'] - output_df[output_df['file'] == reference_run]['width'].mean()) / output_df[output_df['file'] == reference_run]['width'].std()
run_df['depth'] = (run_df['depth'] - output_df[output_df['file'] == reference_run]['depth'].mean()) / output_df[output_df['file'] == reference_run]['depth'].std()
run_df['tracesum'] = (run_df['tracesum'] - output_df[output_df['file'] == reference_run]['tracesum'].mean()) / output_df[output_df['file'] == reference_run]['tracesum'].std()
run_df['tracemax'] = (run_df['tracemax'] - output_df[output_df['file'] == reference_run]['tracemax'].mean()) / output_df[output_df['file'] == reference_run]['tracemax'].std()
run_df['tracedev'] = (run_df['tracedev'] - output_df[output_df['file'] == reference_run]['tracedev'].mean()) / output_df[output_df['file'] == reference_run]['tracedev'].std()
run_df['padnum'] = (run_df['padnum'] - output_df[output_df['file'] == reference_run]['padnum'].mean()) / output_df[output_df['file'] == reference_run]['padnum'].std()

# sum of absolute deviations from reference run
run_df['Deviation'] = abs(run_df['length']) + abs(run_df['width']) + abs(run_df['depth']) + abs(run_df['tracesum']) + abs(run_df['tracemax']) + abs(run_df['tracedev']) + abs(run_df['padnum'])

In [17]:
run_df = run_df.sort_values(by=['Deviation'])
best_run_df = run_df.head(10)

# load in parameters for best runs
params = pd.read_csv('C:\\Users\\Adam\\OneDrive - Tenundra, Inc\\Desktop\\Programming\\GADGET2\\simInput\\parameters.csv')

best_params = params[params['Sim'].isin(best_run_df['file'].str.replace('.h5',''))]

In [18]:
parameters = ['Threshold', 'EIonize', 'Fano', 'CoefL', 'CoefT', 'Gain', 'GETGain', 'PeakingTime']
print('Current Best Fit Parameters')
for p in parameters:
    print(p, '\t', best_params[p].mean(),'\t', best_params[p].std()/best_params[p].mean(), '\n', params[p].mean(),'\t', params[p].std()/params[p].mean(), '\n')

Current Best Fit Parameters
Threshold 	 24.0 	 0.11410886614690961 
 23.14 	 0.18447695837259964 

EIonize 	 30.699999999999996 	 0.052066343202429255 
 29.032999999999998 	 0.09611345465862343 

Fano 	 0.21777777777777776 	 0.10957516119854353 
 0.23039999999999997 	 0.09810847212397264 

CoefL 	 0.00011988888888888887 	 0.09036144578313257 
 0.00011605000000000003 	 0.08570173056346798 

CoefT 	 0.002117777777777778 	 0.06533426310760941 
 0.0021636 	 0.09108053825457481 

Gain 	 8634.555555555555 	 0.07749923483742906 
 9655.427000000001 	 0.09930537623198606 

GETGain 	 135.47777777777776 	 0.08542364052239486 
 126.65299999999999 	 0.09675924127436898 

PeakingTime 	 595.7777777777778 	 0.10086864085274529 
 686.85 	 0.09744132862545504 



In [19]:
print('Full Run Deviation:', run_df['Deviation'].mean(), '\t', run_df['Deviation'].std()/run_df['Deviation'].mean())
print('Best Run Deviation:', best_run_df['Deviation'].mean(), '\t', best_run_df['Deviation'].std()/best_run_df['Deviation'].mean())

Full Run Deviation: 168.6771112084371 	 0.3448872344462877
Best Run Deviation: 67.94220111334235 	 0.42770743679567547


In [21]:
best_run_df.head()

,file,length,width,depth,tracesum,tracemax,tracedev,padnum,Deviation
0,273_800p.h5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
49,tuning2protons52-800p.h5,2.866052,2.850431,5.757850,21.853772,0.307368,1.065530,2.660329,37.361332
67,tuning2protons69-800p.h5,2.030567,2.445490,5.618571,42.473774,0.636950,2.083717,2.665179,57.954246
43,tuning2protons47-800p.h5,4.676599,3.532785,7.026455,53.141784,1.250347,3.093783,1.758340,74.480093
60,tuning2protons62-800p.h5,3.650963,3.193452,5.604332,55.736909,1.630807,3.610461,4.056958,77.483882
